In [24]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from ast import literal_eval
from functools import reduce
from IPython.display import display, Markdown

In [2]:
sample_submission=pd.read_csv("../data/raw/sample_submission.csv")
specs=pd.read_csv("../data/raw/specs.csv")
test=pd.read_csv("../data/raw/test.csv")
train=pd.read_csv("../data/raw/train.csv")
train_labels=pd.read_csv("../data/raw/train_labels.csv")

In [3]:
train.event_code.nunique()*train.title.nunique(), specs.event_id

(1848, 0      2b9272f4
 1      df4fe8b6
 2      3babcb9b
 3      7f0836bf
 4      ab3136ba
          ...   
 381    29f54413
 382    06372577
 383    2a444e03
 384    9e6b7fb5
 385    d3640339
 Name: event_id, Length: 386, dtype: object)

In [4]:
combined=pd.concat([train, test])

In [5]:
raw_events="""
skip tutor:
d2659ab4

panbalance:
bc8f2793

morebalance:
804ee27f

bucket:
91561152

jar:
90efca10

bucket resource:
5348fd84

moar buckets:
37ee8496

chkinen/pig:
cdd22e43

anotherchicken:
56bcd38d

crystals:
3d63345e

moar crystals:
9d4e7b25

crystals done:
d122731b

ohfuck crystals:
5c2f29ca

yep crystals:
a1192f43

castles man:
84538528

numbers:
3bb91dda

toy drag:
85de926c

green check:
a5e9da97

hat/birds:
8fee50e2

more hats:
a76029ee

dinos:
d02b7a8e

dinos bldad:
2dc29e21

dinos yahooey:
262136f4

mushroom miss:
0d18d96c

mushroom pick:
fbaf3456

tubfill:
8f094001

chest drag:
3d0b9317

can drop:
598f4598

more dinos:
c0415e5c

dino forever:
b120f2ac

even more dinos:
71e712d8

fuck dinosaurs:
1cc7cfca

cauldrons:
30614231

bowls:
3d8c61b0

crystal moar:
5e109ec3

bowls more:
8af75982

bowl pick:
792530f8

chest:
a8efe47b

caterpillar:
4a4c3d21

go dino:
14de4c5d

chest pillars:
0db6d71d

gates:
461eace6

mushroom:
5f0eb72c

bug:
363c86c9

more bugs:
5e812b27

chest miss:
9ce586dd

amounts:
28f975ea

castle:
c58186bf

hats:
d38c2fd7

icon:
857f21c0

rockit:
e694a35b

food scale:
d185d3ea

hole watering:
74e5f8a7

bowl scale:
6bf9e3e1

bowl miss:
a2df0760

give a hint:
4e5fc6f5
731c0cbe
e080a381
92687c59
08ff79ad
cb1178ad
19967db1

bowlscales:
37db1c2f

mushroom res:
c7128948

done solution:
17113b36
25fa8af4
93b353f2
070a5291
392e14df

bottle miss:
d3f1e122

sand:
9ee1c98c

more rockit:
02a42007

chicken:
499edb7c

scale food:
4ef8cdd3

bird:
4c2ec19f

cancan:
5d042115

bugtape:
022b4259

basket:
fcfdffb6

moretoy:
ad148f58

partial cloud:
f50fc6c1

shelf:
5c3d2b2f

click group:
86c924c4

misses:
bcceccc6
1325467d
a7640a16
bd612267
15a43e5b
f3cd5473
7372e1a5
76babcde
6c517a88
4bb2f698
7dfe6d8a
9e34ea74
c952eb01
3ee399c3
7da34a02
565a3990
a44b10dc
acf5c23f
587b5989
a16a373e
a0faea5d
5e3ea25a

calls help:
93edfe2e
6043a2b4
05ad839b
37937459
6aeafed4
6f8106d9
77c76bc5
f54238ee
47f43a44
85d1b0de
47efca07
eb2c19cd
6f4bd64e
e7e44842
67aa2ada
8d748b58
d3640339

shape:
38074c54

play again:
b1d5101d
3b2048ee
ecc36b7f
46b50ba8
9e6b7fb5

mushroomstart:
db02c830

weight:
2a444e03

dropweight:
e7561dd2

yeabadino:
a6d66e51

bugcick:
e79f3763

carterpillarstart:
51102b85

chestplace:
562cec5f

cloudfill:
d2e9262e

cloudempty:
2fb91ec1

mushroom:
28ed704e

beat round:
56817e2b
28520915
08fd73f3
6c930e6e
b74258a0
e9c52111
f5b8c21a
b012cd7f
00c73085
3323d7e9
ca11f653
895865f3
36fa3ebe
1c178d24
f6947f54
83c6c409

beat level:
37c53127

start level:
dcaede90

bottle:
bb3e370b

pig:
46cd75b4

shellmove:
1996c610

scalesmore:
cfbd47c8


canmf:
de26c3a6

start round:
3dfd4aa4
65a38bf7
736f9581
2c4e6db0
f32856e4
532a2afb
7cf1bc53
b7530680
63f13dd7
155f62a4
26fd2d99
7961e599
f806dc10
1beb320a
ec138c1c
c2baf0bd
a592d54e
1575e76c

ragequit:
a8cc6fec
1b54d27f
b738d3d3
3393b68b
2b058fe3
9565bea6
17ca3959
a5be6304
222660ff
4074bac2
003cd2ee
b2e5b0f1
"""

In [7]:
len(combined[combined.title!="Assessment"].event_id.unique())

384

In [6]:
events = []
cur = []
for e in raw_events.split("\n"):
    if not e.strip():
        continue
    elif ":" in e:
        if cur:
            events.append(cur)
            cur = []
    else:
        cur.append(e.strip())
if cur:
    events.append(cur)

In [7]:
game_events=list(combined[combined.type=="Game"].event_id.unique())

In [8]:
real_events = [x for x in [[e for e in ev if e in game_events] for ev in events] if x]

In [9]:
[e for e in real_events if len(e)==1]

[['d2659ab4'],
 ['bc8f2793'],
 ['804ee27f'],
 ['a1192f43'],
 ['3bb91dda'],
 ['85de926c'],
 ['a5e9da97'],
 ['d02b7a8e'],
 ['2dc29e21'],
 ['262136f4'],
 ['8f094001'],
 ['c0415e5c'],
 ['b120f2ac'],
 ['1cc7cfca'],
 ['3d8c61b0'],
 ['8af75982'],
 ['792530f8'],
 ['14de4c5d'],
 ['28f975ea'],
 ['857f21c0'],
 ['d185d3ea'],
 ['74e5f8a7'],
 ['6bf9e3e1'],
 ['a2df0760'],
 ['37db1c2f'],
 ['4ef8cdd3'],
 ['ad148f58'],
 ['5c3d2b2f'],
 ['86c924c4'],
 ['2a444e03'],
 ['e7561dd2'],
 ['37c53127'],
 ['dcaede90'],
 ['1996c610'],
 ['cfbd47c8']]

In [10]:
correct_events=[e for e in  ['47026d5f', '3afde5dd', '3ddc79c3', '45d01abe', '6f4adc4b', 'd3268efa', '77ead60d', 
                            '9b4001e4', '3afb49e6', '250513af', 'c7fe2a55', '73757a5e', 'e3ff61fb', 
                            'daac11b0', 'a8876db3', '9d29771f'] if e in game_events]
correct_events

['47026d5f',
 '3afde5dd',
 '3ddc79c3',
 '6f4adc4b',
 '77ead60d',
 '9b4001e4',
 '250513af',
 'c7fe2a55',
 '73757a5e',
 'e3ff61fb',
 'daac11b0']

In [8]:
final_events=[
    ['47026d5f',
 '3afde5dd',
 '3ddc79c3',
 '6f4adc4b',
 '77ead60d',
 '9b4001e4',
 '250513af',
 'c7fe2a55',
 '73757a5e',
 'e3ff61fb',
 'daac11b0'],
    ['1cf54632',
 '7d093bf9',
 '8ac7cce4',
 '51311d7a',
 '29bdd9ba',
 'd9c005dd',
 '48349b14',
 '6d90d394',
 '9c5ef70c'],
['d88e8f25',
 'c277e121',
 '7423acbc',
 '1af8be29',
 '763fc34e',
 '90ea0bac',
 'e5734469',
 '9de5e594',
 '907a054b',
 '44cb4907',
 '0330ab6a'],
    ['2b9272f4',
 'e720d930',
 '709b1251',
 '4d911100',
 'cf7638f3',
 'e5c9df6f',
 'a8a78786',
 '55115cbd',
 'cb6010f8',
 '1f19558b',
 '58a0de5c'],
['5859dfb6',
 'e04fb33d',
 '28a4eb9a',
 'e57dd7af',
 '2230fab4',
 'c51d8688',
 '89aace00',
 '8b757ab8',
 'd45ed6a1',
 'ac92046e',
 '3bf1cf26'],
['3babcb9b',
 '7f0836bf',
 'ab3136ba',
 '15f99afc',
 '0d1da71f',
 'f7e47413',
 '69fdac0a',
 '2a512369',
 '9e4c8c7b',
 '15eb4a7d',
 'a1bbe385'],
['33505eae',
 '363d3849',
 'a29c5338',
 '6cf7d25c',
 '7ec0c298',
 '832735e1',
 '5154fc30',
 '0413e89d',
 'f71c4741',
 'f28c589a',
 '8d7e386c'],
['119b5b02',
 '5dc079d8',
 '26a5a3dd',
 '6f445b57',
 '29a42aea',
 '01ca3a3c',
 'e4d32835',
 'ab4ec3a4',
 'dcb1663e',
 '0ce40006'],
['4b5efe37',
 'abc5811c',
 '8d84fa81',
 '65abac75',
 '0086365d',
 '87d743c1',
 'fd20ea40',
 'f93fc684',
 'cc5087a3',
 '5be391b5',
 '7040c096'],
    ['b7dc8128', '15ba1109'],
    ['99abe2bb', 'd51b1749', '5a848010'],
    ['86ba578b', '4d6737eb'],
    ['e080a381', '92687c59', '19967db1'],
    ['c189aaf2', '4a09ace1', '99ea62f3'],
    ['53c6e11a', '9ed8f6da'],
    ['6088b756', 'd88ca108'],
    
['bcceccc6',
 'a7640a16',
 'f3cd5473',
 '7372e1a5',
 '76babcde',
 '6c517a88',
 '7dfe6d8a',
 'cf82af56',
 '587b5989',
 'a0faea5d',
 '5e3ea25a'],
 ['93edfe2e',
 '6043a2b4',
 '05ad839b',
 '6aeafed4',
 '6f8106d9',
 '6f4bd64e',
 '67aa2ada',
 'd3640339'],
 ['b1d5101d', '3b2048ee', 'ecc36b7f', '46b50ba8', '9e6b7fb5'],
 ['56817e2b',
 '16dffff1',
 '08fd73f3',
 'f5b8c21a',
 'b012cd7f',
 '00c73085',
 '3323d7e9',
 'ca11f653',
 '895865f3',
 '36fa3ebe',
 '1c178d24'],
 ['2c4e6db0',
 'f32856e4',
 '7cf1bc53',
 '63f13dd7',
 '26fd2d99',
 '7961e599',
 'f806dc10',
 '1beb320a',
 'c2baf0bd',
 'a592d54e',
 '1575e76c'],
 ['17ca3959', '4074bac2'],
['bc8f2793', 'd02b7a8e', 'd185d3ea', 'a2df0760'],
['d2659ab4'],
 ['804ee27f'],
 ['a1192f43'],
 ['3bb91dda'],
 ['85de926c'],
 ['a5e9da97'],
 ['2dc29e21'],
 ['262136f4'],
 ['8f094001'],
 ['c0415e5c'],
 ['b120f2ac'],
 ['1cc7cfca'],
 ['3d8c61b0'],
 ['8af75982'],
 ['792530f8'],
 ['14de4c5d'],
 ['28f975ea'],
 ['857f21c0'],
 ['74e5f8a7'],
 ['6bf9e3e1'],
 ['37db1c2f'],
 ['4ef8cdd3'],
 ['ad148f58'],
 ['5c3d2b2f'],
 ['86c924c4'],
 ['2a444e03'],
 ['e7561dd2'],
 ['37c53127'],
 ['dcaede90'],
 ['1996c610'],
 ['cfbd47c8'],
 ['86ba578b'],
 ['c1cac9a2', '3bb91ced'],
 ['dcb55a27'],
 ['1340b8d7'],
 ['c54cf6c5'],
 ['d06f75b5'],   
 ['c6971acf', '29f54413', '7d5c30a2', '06372577'],
 ['27253bdc'],
 ['27253bdc'],
 ['77261ab5'],
 ['b2dba42b'],
 ['1bb5fbdb'],
 ['1325467d'],
 ['37937459'],
 ['5e812b27'],
 ['c58186bf'],
 ['9ee1c98c'],
 ['84538528'],
 ['e64e2cfd'],
 ['49ed92e9'],
 ['bd701df8'],
 ['f50fc6c1'],
 ['d2e9262e'],
 ['2fb91ec1'],
 ['c952eb01'],
 ['a6d66e51'],
 ['71e712d8'],
 ['e7e44842'],
 ['4901243f'],
 ['beb0a7b9'],
 ['02a42007'],
 ['e694a35b'],
 ['b88f38da'],
 ['884228c8'],
 ['9b01374f'],
 ['56cd3b43'],
 ['a44b10dc'],
 ['bbfe0445'],
 ['5d042115'],
 ['de26c3a6'],
 ['598f4598'],
 ['fcfdffb6'],
 ['c7f7f0e1'],
 ['0a08139c'],
 ['e79f3763'],
 ['71fe8f75'],
 ['363c86c9'],
 ['022b4259'],
 ['565a3990'],
 ['d2278a3b'],
 ['b7530680'],
 ['67439901'],
 ['bb3e370b'],
 ['df4940d3'],
 ['90efca10'],
 ['d3f1e122'],
 ['e9c52111'],
 ['15a43e5b'],
 ['756e5507'],
 ['ea321fb1'],
 ['84b0e0c8'],
 ['4bb2f698'],
 ['56bcd38d'],
 ['499edb7c'],
 ['cdd22e43'],
 ['46cd75b4'],
 ['85d1b0de'],
 ['8d748b58'],
 ['47efca07'],
 ['5f5b2617'],
 ['9b23e8ee'],
 ['7ab78247'],
 ['736f9581'],
 ['b80e5e84'],
 ['4c2ec19f'],
 ['f54238ee'],
 ['47f43a44'],
 ['461eace6'],
 ['9e34ea74'],
 ['08ff79ad'],
 ['611485c5'],
 ['30df3273'],
 ['3a4be871'],
 ['2ec694de'],
 ['16667cc5'],
 ['7fd1ac25'],
 ['a8cc6fec'],
 ['003cd2ee'],
 ['1b54d27f'],
]

In [9]:
all_events=set([])
for x in final_events:
    all_events|=set(x)
#for c in correct_events:
#    assert c in all_events # correct is captured

In [10]:
combined.type.unique()

array(['Clip', 'Activity', 'Game', 'Assessment'], dtype=object)

In [11]:
[[x] for x in combined[combined.type!='Assessment'].event_id.unique() if x not in all_events]

[]

In [15]:
combined.event_id

0          27253bdc
1          27253bdc
2          77261ab5
3          b2dba42b
4          1bb5fbdb
             ...   
1156409    c74f40cd
1156410    6c930e6e
1156411    a5be6304
1156412    27253bdc
1156413    f56e0afc
Name: event_id, Length: 12497456, dtype: object

In [16]:
for x in []:
    assert x not in all_events

In [17]:
'29f54413' in all_events

True

In [12]:
len(final_events)

143

In [18]:
for e in list(specs.event_id[specs.event_id.isin(game_events)]):
    assert e in all_events, e

In [15]:
df=combined[combined.type=="Game"].groupby("title").event_id.unique()

In [16]:
game_events={}

for title, events in df.to_dict().items():
    events=list(events)
    game_events[title]=[x for x in [[e for e in ev if e in events] for ev in final_events] if x]
    for x in game_events[title]:
        assert len(x)==1, f"{x} {title}"
    game_events[title] = [x[0] for x in game_events[title]]

In [ ]:
world_events = 

In [17]:
game_events

{'Air Show': ['9b4001e4',
  '7423acbc',
  '58a0de5c',
  'e04fb33d',
  'a1bbe385',
  'f28c589a',
  'dcb1663e',
  '65abac75',
  '15ba1109',
  'd88ca108',
  'bcceccc6',
  '6f4bd64e',
  'f5b8c21a',
  '1575e76c',
  'd2659ab4',
  '14de4c5d',
  '28f975ea',
  'dcb55a27',
  '06372577'],
 'All Star Sorting': ['daac11b0',
  'c277e121',
  '1f19558b',
  'd45ed6a1',
  '9e4c8c7b',
  '363d3849',
  '26a5a3dd',
  '4b5efe37',
  'b7dc8128',
  '587b5989',
  '6043a2b4',
  'b1d5101d',
  'ca11f653',
  '2c4e6db0',
  'd02b7a8e',
  '2dc29e21',
  'b120f2ac',
  '1cc7cfca'],
 'Bubble Bath': ['6f4adc4b',
  '1cf54632',
  '90ea0bac',
  '55115cbd',
  '5859dfb6',
  '15eb4a7d',
  '0413e89d',
  '29a42aea',
  '8d84fa81',
  '99abe2bb',
  '99ea62f3',
  'a0faea5d',
  '6aeafed4',
  'ecc36b7f',
  '895865f3',
  '1beb320a',
  '3bb91dda',
  '85de926c',
  '8f094001',
  '857f21c0',
  'ad148f58',
  '1340b8d7',
  'c54cf6c5',
  'd06f75b5'],
 'Chow Time': ['47026d5f',
  '7d093bf9',
  '0330ab6a',
  'cb6010f8',
  '2230fab4',
  '0d1da71f',

In [19]:
print("total number of features",len(final_events) + sum(len(v)*len(v)+len(v) for v in game_events.values()))

total number of features 4309


In [20]:
import json 
def hascorrect(x):
    for v in json.loads(x):
        if v["name"]=="correct":
            return True
    return False

specs[specs["args"].apply(hascorrect)].event_id.unique()

array(['8fee50e2', '30614231', '5f0eb72c', 'd122731b', 'a5e9da97',
       '0db6d71d', '91561152', '14de4c5d', '8af75982', '28f975ea',
       'c0415e5c', '3bb91dda', '17113b36', '25fa8af4', '93b353f2',
       '4a4c3d21', '4ef8cdd3', '2dc29e21', '74e5f8a7', '262136f4',
       '804ee27f', '070a5291', '392e14df', '5c3d2b2f', '86c924c4',
       'e7561dd2'], dtype=object)